In [9]:
# Datos: https://datosabiertos.bogota.gov.co/dataset/8c2a07f9-5948-41ca-8f7a-447753426ab9/resource/4911565b-4518-4931-93de-d117c10dbdce/download/osb_ocupacion_hosp_ucis.csv

In [1]:
import pandas as pd

In [2]:
camas = pd.read_csv( './data/camas_bogota_raw.csv', encoding = 'ISO-8859-1', dtype = { 'Camas Ocupadas': int, 'Camas habilitadas': int, 'Camas disponibles': int } )

In [3]:
camas.dtypes

IPS                        object
Servicio                   object
Grupo                      object
Camas Ocupadas              int64
Camas habilitadas           int64
Camas disponibles           int64
Porcentaje de Ocupación    object
Fecha Actualización        object
dtype: object

In [4]:
camas

,IPS,Servicio,Grupo,Camas Ocupadas,Camas habilitadas,Camas disponibles,Porcentaje de Ocupación,Fecha Actualización
0,Públicas,Hospitalización General,Neonatal,104,138,34,75%,31/05/2020
1,Públicas,Hospitalización General,Pediátrico,103,400,297,26%,31/05/2020
2,Públicas,Hospitalización General,Adulto,985,1188,387,72%,31/05/2020
3,Públicas,Unidad de Cuidado Intermedio,Neonatal,92,126,34,73%,31/05/2020
4,Públicas,Unidad de Cuidado Intermedio,Pediátrico,6,14,8,43%,31/05/2020
5,Públicas,Unidad de Cuidado Intermedio,Adulto,56,72,16,78%,31/05/2020
6,Públicas,Unidad de cuidado intensivo,Neonatal,58,68,10,85%,31/05/2020
7,Públicas,Unidad de cuidado intensivo,Pediátrico,13,29,16,45%,31/05/2020
8,Públicas,Unidad de cuidado intensivo,Adulto,118,137,45,72%,31/05/2020
9,Privadas,Hospitalización General,Neonatal,102,149,47,68%,31/05/2020


In [5]:
camas_grouped = camas.groupby( 'Servicio' ).agg( { 'Camas Ocupadas': 'sum', 'Camas habilitadas': 'sum', 'Camas disponibles': 'sum' } ).reset_index()

In [6]:
camas_grouped[ '% Ocupación' ] = camas_grouped[ 'Camas Ocupadas' ] / camas_grouped[ 'Camas habilitadas' ]

In [7]:
camas_grouped

,Servicio,Camas Ocupadas,Camas habilitadas,Camas disponibles,% Ocupación
0,Hospitalización General,5286,8146,3175,0.648907
1,Unidad de Cuidado Intermedio,619,957,383,0.646813
2,Unidad de cuidado intensivo,993,1406,635,0.706259


In [8]:
camas_grouped.to_csv( './data/camas_bogota.csv', index = False )